In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')

In [ ]:
answer = pd.read_csv('submission_example.csv')

In [ ]:
type(answer)

In [ ]:
answer.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train_labels = df_train['mean_exam_points'].values
df_train_data = df_train.drop('mean_exam_points', axis=1).values
df_test_data = df_test.values

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(df_train_data, 
                                                                    df_train_labels, 
                                                                    test_size = 0.3,
                                                                    random_state = 1)

In [ ]:
def get_bootstrap(data, labels, N):
    n_samples = data.shape[0]
    bootstrap = []
    
    for i in range(N):
        b_data = np.zeros(data.shape)
        b_labels = np.zeros(labels.shape)
        
        for j in range(n_samples):
            sample_index = np.random.randint(0, n_samples-1)
            b_data[j] = data[sample_index]
            b_labels[j] = labels[sample_index]
        bootstrap.append((b_data, b_labels))
        
    return bootstrap

In [ ]:
def get_subsample(len_sample):
    # будем сохранять не сами признаки, а их индексы
    sample_indexes = [i for i in range(len_sample)]
    
    len_subsample = int(np.sqrt(len_sample))
    subsample = []
    
    np.random.shuffle(sample_indexes)
    for _ in range(len_subsample):
        subsample.append(sample_indexes.pop())
        
    return subsample

In [ ]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [ ]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет среднего значения по выборке
        prediction = np.mean(self.labels)
        
        return prediction        

In [ ]:
# Критерий информативности

def dispersion(labels):
    impurity = 0
    labels_mean = np.mean(labels)
    
    for label in labels:
        impurity += (label - labels_mean) ** 2
    
    impurity = impurity / len(labels)
        
    return impurity

In [ ]:
# Расчет качества

def quality(left_labels, right_labels, current_dispersion):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_dispersion - p * dispersion(left_labels) - (1 - p) * dispersion(right_labels)

In [17]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [18]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 1

    current_dispersion = dispersion(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    # выбор индекса из подвыборки длиной sqrt(n_features)
    subsample = get_subsample(n_features)
    
    for index in subsample:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_dispersion)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [19]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, depth=1, max_depth=9):

    quality, t, index = find_best_split(data, labels)
    
    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0 or depth == max_depth:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, depth+1, max_depth)
    false_branch = build_tree(false_data, false_labels, depth+1, max_depth)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [20]:
def random_forest(data, labels, n_trees, depth=9):
    forest = []
    bootstrap = get_bootstrap(data, labels, n_trees)
    
    for b_data, b_labels in bootstrap:
        forest.append(build_tree(b_data, b_labels, max_depth=depth))
        
    return forest

In [21]:
# Функция регрессии для отдельного объекта

def reg_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return reg_object(obj, node.true_branch)
    else:
        return reg_object(obj, node.false_branch)

In [22]:
# функция формирования предсказания по выборке на одном дереве

def predict(data, tree):
    
    reg = []
    for obj in data:
        prediction = reg_object(obj, tree)
        reg.append(prediction)
    return reg

In [23]:
# предсказание голосованием деревьев

def tree_vote(forest, data):

    # добавим предсказания всех деревьев в список
    predictions = []
    for tree in forest:
        predictions.append(predict(data, tree))
    
    # сформируем список с предсказаниями для каждого объекта
    predictions_per_object = list(zip(*predictions))
    
    # выберем в качестве итогового предсказания для каждого объекта среднее значение по предсказаниям
    voted_predictions = []
    for obj in predictions_per_object:
        voted_predictions.append(np.mean(obj))
        
    return voted_predictions

In [24]:
def R_2(actual, predicted):
    ESS = 0
    TSS = 0
    
    y_mean = np.mean(actual)
    
    for y_i, a_i in zip(actual, predicted):
        ESS += (a_i - y_i) ** 2
        TSS += (y_i - y_mean) ** 2
         
    return 1 - ESS/TSS

In [ ]:
max_depth_list = [9, 10]
n_tree_list = [10, 20, 50]

In [ ]:
for max_depth in max_depth_list:
    print(max_depth, '\n')
    for n_trees in n_tree_list:
        print(n_trees)
        my_forest_1 = random_forest(train_data, train_labels, n_trees, depth=max_depth)
        
        # Получим ответы для обучающей выборки 
        train_answers = tree_vote(my_forest_1, train_data)
        
        # Получим ответы для обучающей выборки 
        train_answers = tree_vote(my_forest_1, train_data)
        
        # И получим ответы для тестовой выборки
        test_answers = tree_vote(my_forest_1, test_data)
        
        # Точность на обучающей выборке
        train_R_2 = R_2(train_labels, train_answers)
        print(f'Точность случайного леса из {n_trees} деревьев на обучающей выборке: {train_R_2:.3f}')
        
        # Точность на тестовой выборке
        test_R_2 = R_2(test_labels, test_answers)
        print(f'Точность случайного леса из {n_trees} деревьев на тестовой выборке: {test_R_2:.3f}')

In [29]:
n_trees = 200
max_depth = 9

my_forest_200 = random_forest(train_data, train_labels, n_trees, depth=max_depth)
        
# Получим ответы для обучающей выборки 
train_answers = tree_vote(my_forest_200, train_data)

# Получим ответы для обучающей выборки 
train_answers = tree_vote(my_forest_200, train_data)

# И получим ответы для тестовой выборки
test_answers = tree_vote(my_forest_200, test_data)

# Точность на обучающей выборке
train_R_2 = R_2(train_labels, train_answers)
print(f'R_2 случайного леса из {n_trees} деревьев на обучающей выборке: {train_R_2:.3f}')

# Точность на тестовой выборке
test_R_2 = R_2(test_labels, test_answers)
print(f'R_2 случайного леса из {n_trees} деревьев на тестовой выборке: {test_R_2:.3f}')

R_2 случайного леса из 200 деревьев на обучающей выборке: 0.778
R_2 случайного леса из 200 деревьев на тестовой выборке: 0.744


In [25]:
n_trees = 100
max_depth = 9

my_forest_100 = random_forest(train_data, train_labels, n_trees, depth=max_depth)
        
# Получим ответы для обучающей выборки 
train_answers = tree_vote(my_forest_100, train_data)

# Получим ответы для обучающей выборки 
train_answers = tree_vote(my_forest_100, train_data)

# И получим ответы для тестовой выборки
test_answers = tree_vote(my_forest_100, test_data)

# Точность на обучающей выборке
train_R_2 = R_2(train_labels, train_answers)
print(f'R_2 случайного леса из {n_trees} деревьев на обучающей выборке: {train_R_2:.3f}')

# Точность на тестовой выборке
test_R_2 = R_2(test_labels, test_answers)
print(f'R_2 случайного леса из {n_trees} деревьев на тестовой выборке: {test_R_2:.3f}')

R_2 случайного леса из 100 деревьев на обучающей выборке: 0.781
R_2 случайного леса из 100 деревьев на тестовой выборке: 0.749


In [26]:
answer['mean_exam_points'] = tree_vote(my_forest_100, df_test_data)

In [27]:
answer.head()

,Id,mean_exam_points
0,10000,55.247860
1,10001,64.244360
2,10002,53.926562
3,10003,89.009754
4,10004,88.220673


In [28]:
answer.to_csv('submission.csv', index=None)